
<h2> Treatment Pathways Network Study Using Strategus </h2>

<h4> Step 1 - Import the necessary libraries </h4>


In [ ]:
%%jupyter
# import libraries
library(rD2E)
library(Strategus)
library(dplyr)

<h4> Step 2 - Cohort definition loading </h4>

In [ ]:
%%jupyter
cohorts_set <- c(13)
cohortDefinitionSet <- rD2E::get_cohort_definition_set(cohort_set)

<h4> Step 3 - Define the network study components </h4>

In [ ]:
%%jupyter
# Study period
studyStartDate < -"19001201"
studyEndDate < -"20231231"

# Assume target cohort is for depression
targetCohortId <- 1  # actual depression cohort ID

# Event cohorts: SSRI, SNRI, TCA, bupropion, esketamine
eventCohortIds <- c(2, 3, 4, 5, 6)  

# Shared resources
cgModuleSettingsCreator <- CohortGeneratorModule$new()
cohortDefinitionShared <- cgModuleSettingsCreator$createCohortSharedResourceSpecifications(cohortDefinitionSet)
cohortGeneratorModuleSpecifications <- cgModuleSettingsCreator$createModuleSpecifications(generateStats = TRUE)

In [ ]:
%%Jupyter
# TreatmentPatterns module setup
tpModuleSettingsCreator <- TreatmentPatternsModule$new()

treatmentPatternsSpecifications <- tpModuleSettingsCreator$createModuleSpecifications(
  targetCohortId = targetCohortId,
  eventCohortIds = eventCohortIds,
  studyWindow = list(
    startDate = studyStartDate,
    endDate = studyEndDate
  ),
  periodPriorToIndex = 0,
  minEraDuration = 0,
  combinationWindow = 30,
  minPostCombinationDuration = 0,
  splitEventCohorts = TRUE,
  minCellCount = 5,
  groupCombinations = TRUE
)

In [ ]:
%%jupyter
# Create analysis spec
analysisSpecifications <- Strategus::createEmptyAnalysisSpecificiations() |>
  Strategus::addSharedResources(cohortDefinitionShared) |>
  Strategus::addModuleSpecifications(cohortGeneratorModuleSpecifications) |>
  Strategus::addModuleSpecifications(treatmentPatternsSpecifications)

# Save JSON
#ParallelLogger::saveSettingsToJson(
#  analysisSpecifications,
#  file.path("inst", "Eunomia", "SampleStudy", "treatmentPatternsAnalysisSpecification.json")
#)

<h4> Step 4 - Execute the Strategus study </h4>

In [ ]:
%%jupyter
study_name <- "treatment_pathways_study"  # Unique study name
options <- create_options(upload_results=TRUE, study_id = study_name) # set a study_id with a unique id
rD2E::run_strategus_flow(analysisSpecification = analysisSpecifications, options = options)